In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [10]:
def import_data():
    train_data = pd.read_csv('titanic/train.csv')
    test_data = pd.read_csv('titanic/test.csv')
    return train_data, test_data

def load_data(enable_labels=None):
    train_data, test_data = import_data()
    if not enable_labels:
        labels = train_data.columns.value
        train_x = labels.delete('Survived')
        test_x = test_data[enable_labels]
    else:
        train_x = train_data[enable_labels]
        test_x = test_data[enable_labels]
    
    train_y = train_data['Survived']
    test_y = test_data['Survived']
    
    return (train_x, train_y), (test_x, test_y)


In [3]:
train_data, test_data = load_data(['Fare'])
train_data.head()
test_data.head()

train_desc = train_data.describe()
test_desc = test_data.describe()


KeyError: 'Survived'

In [12]:
def null_table(df): 
        null_val = df.isnull().sum()
        percent = 100 * df.isnull().sum()/len(df)
        kesson_table = pd.concat([null_val, percent], axis=1)
        kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
        return kesson_table_ren_columns

null_train = null_table(train_data)
null_test = null_table(test_data)


NameError: name 'train_data' is not defined

In [ ]:
class LNN(nn.modules):
    def __init__(self, input_dim, output_dim, depth):
        self.fc_input = nn.Linear(input_dim, 256)
        self.fc_array = nn.ModuleList([nn.Linear(256, 256) for _ in range(depth - 2)])
        self.fc_output = nn.Linear(256, output_dim)

    def normal_fwd(self, x, training=False):
        y = F.relu(self.fc_input(x))
        for layer in self.fc_array:
            y = F.relu(layer(y))
        y = self.fc_output(y)
        return y

model = LNN(128, 1, 16)
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

for i in range(100):
  # 順伝播
  out = model(input)
  # ロスの計算
  loss = criterion(out, target)
  # 勾配の初期化
  optimizer.zero_grad()
  # 勾配の計算
  loss.backward()
  # パラメータの更新
  optimizer.step()

In [33]:
train, test = import_data()
train['fams'] = train['Parch'] + train['SibSp'] + 1
train['IsAlone'] = 0
train.loc[train['fams'] == 1, 'IsAlone'] = 1
train['Age*Class'] = train.Age * train.Pclass
embarkation_mode = train['Embarked'].dropna().mode()[0]
train['Embarked'] =train['Embarked'].fillna(embarkation_mode)

gender_mapping = {'male': 1, 'female': 0}
train['Sex'] = train['Sex'].map(gender_mapping)
print(train['Age'].isnull())
# train.corr()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888     True
889    False
890    False
Name: Age, Length: 891, dtype: bool
